In [1]:
import numpy as np
import os
import torch
import torch.nn as nn 
from datetime import datetime
from sklearn import preprocessing 
import pickle 
import scipy.io as sio 
from TraditionalFusion.utils import generating_data, svm_classification
import time 


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [9]:
os.environ['FILE_PATH'] = '../SEED/SEED_EEG/'
FILE_PATH = os.getenv('FILE_PATH', '/mnt/g/Meu Drive/mestrado_ppget')
print(FILE_PATH)
eeg_dir = os.path.join(FILE_PATH,"SEED_EEG/ExtractedFeatures/")
eeg_file_list = os.listdir(eeg_dir)
eeg_file_list.sort()


feature_list = ['de_LDS']


../SEED/SEED_EEG/


In [3]:
print(eeg_file_list)

['10_20131130.mat', '10_20131204.mat', '10_20131211.mat', '11_20140618.mat', '11_20140625.mat', '11_20140630.mat', '12_20131127.mat', '12_20131201.mat', '12_20131207.mat', '13_20140527.mat', '13_20140603.mat', '13_20140610.mat', '14_20140601.mat', '14_20140615.mat', '14_20140627.mat', '15_20130709.mat', '15_20131016.mat', '15_20131105.mat', '1_20131027.mat', '1_20131030.mat', '1_20131107.mat', '2_20140404.mat', '2_20140413.mat', '2_20140419.mat', '3_20140603.mat', '3_20140611.mat', '3_20140629.mat', '4_20140621.mat', '4_20140702.mat', '4_20140705.mat', '5_20140411.mat', '5_20140418.mat', '5_20140506.mat', '6_20130712.mat', '6_20131016.mat', '6_20131113.mat', '7_20131027.mat', '7_20131030.mat', '7_20131106.mat', '8_20140511.mat', '8_20140514.mat', '8_20140521.mat', '9_20140620.mat', '9_20140627.mat', '9_20140704.mat', 'label.mat', 'readme.txt']


In [12]:
item = eeg_file_list[0]
fn = feature_list[0]
label_list = np.array([2,1,0,0,1,2,0,1,2,2,1,0,1,2,0]) 


cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")


- reading file: 10_20131130.mat
--user:10, sessions: 2013-11-30 00:00:00


In [6]:
all_data = sio.loadmat(os.path.join(eeg_dir, item))

In [29]:
def generating_data(data_dict, clip_label, feature_name):
    # first 9 movies as training, the last 6 movies as testing
    train_data = data_dict[feature_name+'1']
    print("\t### TRAIN ###")
    print(f"\t\t{feature_name + '1'} - video 1 - {train_data.shape} observacoes")
    _, num, _ = train_data.shape 
    train_label = np.zeros(num,) + clip_label[0]
    train_data = np.swapaxes(train_data, 0, 1)
    train_data = np.reshape(train_data, (num, -1))
    train_residual_index = [2,3,4,5,6,7,8,9]
    for ind,i in enumerate(train_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+1]
        used_data = np.swapaxes(used_data, 0, 1)
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        train_data = np.vstack((train_data, used_data))
        train_label = np.hstack((train_label, used_label))
    
    test_data = data_dict[feature_name+'10']
    print("\t### TEST ###")
    print(f"\t\t{feature_name + '10'} - video 10 - {test_data.shape} observacoes")
    _, num, _ = test_data.shape 
    test_label = np.zeros(num,) + clip_label[9]
    test_data = np.swapaxes(test_data, 0, 1)
    test_data = np.reshape(test_data, (num, -1))
    test_residual_index = [11,12,13,14,15]
    for ind,i in enumerate(test_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+9]
        used_data = np.swapaxes(used_data, 0, 1)
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        test_data = np.vstack((test_data, used_data))
        test_label = np.hstack((test_label, used_label))
    return train_data, test_data, train_label, test_label

In [31]:
train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)


	### TRAIN ###
		de_LDS1 - video 1 - (62, 235, 5) observacoes
		de_LDS2 - video 2 - (233, 310) observacoes
		de_LDS3 - video 3 - (206, 310) observacoes
		de_LDS4 - video 4 - (238, 310) observacoes
		de_LDS5 - video 5 - (185, 310) observacoes
		de_LDS6 - video 6 - (195, 310) observacoes
		de_LDS7 - video 7 - (237, 310) observacoes
		de_LDS8 - video 8 - (216, 310) observacoes
		de_LDS9 - video 9 - (265, 310) observacoes
	### TEST ###
		de_LDS10 - video 10 - (62, 237, 5) observacoes
		de_LDS11 - video 11 - (235, 310) observacoes
		de_LDS12 - video 12 - (233, 310) observacoes
		de_LDS13 - video 13 - (235, 310) observacoes
		de_LDS14 - video 14 - (238, 310) observacoes
		de_LDS15 - video 15 - (206, 310) observacoes


In [16]:
train_data.shape

(2010, 310)

In [22]:
train_data

array([[27.08358157, 22.91154056, 21.15346081, ..., 20.9950124 ,
        19.32675924, 18.13914969],
       [27.08847498, 22.90886303, 21.15102226, ..., 20.99591001,
        19.32718201, 18.13893663],
       [27.09624093, 22.90799629, 21.15015659, ..., 20.99704567,
        19.32747367, 18.13900468],
       ...,
       [26.46839952, 22.68214713, 21.12216414, ..., 20.80866727,
        19.17956115, 18.15544733],
       [26.46748892, 22.68504015, 21.12516397, ..., 20.80912973,
        19.17944841, 18.1553233 ],
       [26.46881893, 22.68545062, 21.12657624, ..., 20.80823875,
        19.17961697, 18.15547013]])

verificando com dado original para ver se faz sentido

In [19]:
all_data[fn+'1']

array([[[27.08358157, 22.91154056, 21.15346081, 19.74299538,
         19.15590487],
        [27.08847498, 22.90886303, 21.15102226, 19.74405133,
         19.15741419],
        [27.09624093, 22.90799629, 21.15015659, 19.74556429,
         19.15839686],
        ...,
        [26.3270917 , 22.77351568, 21.02755558, 19.48076774,
         18.81533067],
        [26.32816947, 22.77516187, 21.02765953, 19.48027935,
         18.81582815],
        [26.32750318, 22.77408488, 21.02622243, 19.47979029,
         18.81601386]],

       [[27.17584979, 22.94684656, 21.14874759, 18.96109104,
         17.88738954],
        [27.17383851, 22.9448068 , 21.14691101, 18.96060752,
         17.88768093],
        [27.17275625, 22.94427799, 21.14620135, 18.96089588,
         17.8874079 ],
        ...,
        [27.49219772, 22.81414286, 20.91274106, 18.91500009,
         17.85497408],
        [27.49042437, 22.8150694 , 20.91244147, 18.91393177,
         17.85521387],
        [27.48979895, 22.81340214, 20.91095994, 

In [32]:
data_to_validate = all_data[fn+'1']

print(data_to_validate.shape)
EEG_CHANEL = 0
print(data_to_validate[EEG_CHANEL].shape)
data_to_validate[EEG_CHANEL]


(62, 235, 5)
(235, 5)


array([[27.08358157, 22.91154056, 21.15346081, 19.74299538, 19.15590487],
       [27.08847498, 22.90886303, 21.15102226, 19.74405133, 19.15741419],
       [27.09624093, 22.90799629, 21.15015659, 19.74556429, 19.15839686],
       ...,
       [26.3270917 , 22.77351568, 21.02755558, 19.48076774, 18.81533067],
       [26.32816947, 22.77516187, 21.02765953, 19.48027935, 18.81582815],
       [26.32750318, 22.77408488, 21.02622243, 19.47979029, 18.81601386]])

In [39]:
all_eeg_channel, num, bands = data_to_validate.shape 

data_to_validate_1 = np.swapaxes(data_to_validate, 0, 1)
data_to_validate_1.shape

(235, 62, 5)

In [40]:
data_to_validate_1[0]

array([[27.08358157, 22.91154056, 21.15346081, 19.74299538, 19.15590487],
       [27.17584979, 22.94684656, 21.14874759, 18.96109104, 17.88738954],
       [26.9454401 , 22.86094665, 21.17235922, 20.20211872, 19.77274198],
       [26.56752832, 21.91597884, 20.35183976, 18.90549729, 18.00130377],
       [26.35195414, 22.06651715, 20.52819908, 19.54897846, 18.88284228],
       [26.41221777, 21.99052302, 20.67284757, 19.30904259, 18.71435661],
       [25.88998838, 21.44124289, 19.99019842, 18.61154119, 17.64579834],
       [26.18808427, 21.068819  , 19.65305701, 18.14251408, 16.91859981],
       [26.64775298, 21.00490243, 19.55352221, 17.86204289, 16.40316211],
       [26.58222148, 21.07121585, 19.57163451, 17.93796315, 16.37941519],
       [26.49185029, 21.15787018, 19.69663829, 18.17822358, 16.93941164],
       [25.5060435 , 21.18866263, 19.77884258, 18.8673628 , 18.0678937 ],
       [27.72421883, 21.73711092, 20.27132907, 19.93191388, 19.62630703],
       [28.06378454, 21.96185449, 20.4

In [43]:
data_to_validate_2 = np.reshape(data_to_validate_1, (num, -1))
data_to_validate_2.shape

(235, 310)

In [44]:
data_to_validate_2[0]

array([27.08358157, 22.91154056, 21.15346081, 19.74299538, 19.15590487,
       27.17584979, 22.94684656, 21.14874759, 18.96109104, 17.88738954,
       26.9454401 , 22.86094665, 21.17235922, 20.20211872, 19.77274198,
       26.56752832, 21.91597884, 20.35183976, 18.90549729, 18.00130377,
       26.35195414, 22.06651715, 20.52819908, 19.54897846, 18.88284228,
       26.41221777, 21.99052302, 20.67284757, 19.30904259, 18.71435661,
       25.88998838, 21.44124289, 19.99019842, 18.61154119, 17.64579834,
       26.18808427, 21.068819  , 19.65305701, 18.14251408, 16.91859981,
       26.64775298, 21.00490243, 19.55352221, 17.86204289, 16.40316211,
       26.58222148, 21.07121585, 19.57163451, 17.93796315, 16.37941519,
       26.49185029, 21.15787018, 19.69663829, 18.17822358, 16.93941164,
       25.5060435 , 21.18866263, 19.77884258, 18.8673628 , 18.0678937 ,
       27.72421883, 21.73711092, 20.27132907, 19.93191388, 19.62630703,
       28.06378454, 21.96185449, 20.46994646, 19.92977301, 19.57

aparentemente tudo ok!

# Treinando SVM

In [54]:
print(item)
c = 2

clf = svm.LinearSVC(C=2**c)
clf.fit(train_data, train_label)    
p_labels = clf.predict(test_data)


from sklearn import metrics
score = clf.score(test_data, test_label)
print(f"SCORE: {score}")
print("Accuracy:",metrics.accuracy_score(test_label, p_labels))
# print("Precision:",metrics.precision_score(test_label, p_labels))
# print("Recall:",metrics.recall_score(test_label, p_labels))





10_20131130.mat
SCORE: 0.4060693641618497
Accuracy: 0.4060693641618497


In [56]:
test_data.shape

(1384, 310)

In [60]:
test_label[472:705]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [62]:
p_labels[472:705]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.